In [1]:
from ctypes import *
cdll.LoadLibrary('deps/sparse_rrt/deps/trajopt/build/lib/libsco.so')

<CDLL 'deps/sparse_rrt/deps/trajopt/build/lib/libsco.so', handle 5585e921ef60 at 0x7fdb18b508d0>

In [2]:
from sparse_rrt.planners import SST
from env.cartpole_obs import CartPoleObs
from env.cartpole import CartPole
from sparse_rrt.systems import standard_cpp_systems
from sparse_rrt import _sst_module
import numpy as np
import time
from tools.pcd_generation import rectangle_pcd

system = standard_cpp_systems.CartPole()

In [3]:

#obs_list = np.array(obs_list)
#system = standard_cpp_systems.CartPoleObs(obs_list, 4.)
#system = CartPoleObs(obs_list)
# Create SST planner
min_time_steps = 10
max_time_steps = 200
integration_step = 0.002
max_iter = 100
goal_radius=1.5
random_seed=0
sst_delta_near=2.0
sst_delta_drain=1.2

low = []
high = []
state_bounds = system.get_state_bounds()
for i in range(len(state_bounds)):
    low.append(state_bounds[i][0])
    high.append(state_bounds[i][1])
    
start = np.random.uniform(low=low, high=high)
end = np.random.uniform(low=low, high=high)


start[1] = 0.
start[3] = 0.
end[1] = 0.
end[3] = 0.
planner = SST(
    state_bounds=system.get_state_bounds(),
    control_bounds=system.get_control_bounds(),
    distance=system.distance_computer(),
    start_state=start,
    goal_state=end,
    goal_radius=goal_radius,
    random_seed=0,
    sst_delta_near=sst_delta_near,
    sst_delta_drain=sst_delta_drain
)

    
# Run planning and print out solution is some statistics every few iterations.
time0 = time.time()
for iteration in range(max_iter):
    if iteration % 50 == 0:
        # from time to time use the goal
        sample = end
        planner.step_with_sample(system, sample, min_time_steps, max_time_steps, integration_step)
    else:
        sample = np.random.uniform(low=low, high=high)
        new_sample = planner.step_with_sample(system, sample, min_time_steps, max_time_steps, integration_step)
        print('iteration: %d' % (iteration))
        print('sample:')
        print(sample)
        print('new_sample:')
        print(new_sample)
        # interation: 0.002
        #planner.step_with_sample(system, sample, 2, 20, 0.01)
    
    #if iteration % 100 == 0:
solution = planner.get_solution()
print("Solution: %s, Number of nodes: %s" % (planner.get_solution(), planner.get_number_of_nodes()))

print('time spent: %f' % (time.time() - time0))
assert solution is not None

iteration: 1
sample:
[-0.48476446 -9.02759446  2.74181821  1.62247629]
new_sample:
[-8.6817178  -0.34896474  1.76448319 -1.57889899]
iteration: 2
sample:
[ 15.4180958  -36.71703237  -1.95776907   1.57942446]
new_sample:
[-29.89756294   0.65802059   0.15162959  -1.97972592]
iteration: 3
sample:
[ 11.35356928 -24.40815915   0.87195903  -1.68428294]
new_sample:
[-30.          -6.29445386  -0.68893749   1.40539494]
iteration: 4
sample:
[  1.45544252 -14.28560405   2.59127159  -1.54514807]
new_sample:
[30.         15.57495546  1.02991638 -0.32103253]
iteration: 5
sample:
[26.57058142 -1.67221859 -1.82169202  1.10198009]
new_sample:
[29.47764684  0.2579006   0.9174152   0.86150213]
iteration: 6
sample:
[ 3.05336458e+00  2.35894767e+01  1.06341088e+00 -2.31600329e-02]
new_sample:
[-30.          -7.21269882  -0.71063533  -1.3624954 ]
iteration: 7
sample:
[-12.97241734 -14.28229995   1.90024683  -1.04054693]
new_sample:
[30.         18.55904747  0.47260746  1.70482501]
iteration: 8
sample:
[ 23

AssertionError: 

In [ ]:
states, actions, costs = solution
print(states.shape)
print(actions.shape)

In [ ]:
# visualize the path
"""
Given a list of states, render the environment
"""
from numpy import sin, cos
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import matplotlib.animation as animation
import matplotlib as mpl
import matplotlib.patches as patches
from IPython.display import HTML
from visual.visualizer import Visualizer



class CartPoleVisualizer(Visualizer):
    def __init__(self, system, params):
        super(CartPoleVisualizer, self).__init__(system, params)
        self.dt = 0.05

    def _init(self):
        # clear the current ax
        ax = plt.gca()
        ax.clear()
        # add patches
        state = self.states[0]
        self.pole = patches.Rectangle((state[0]-self.params['pole_w']/2,self.params['cart_h']),\
                                       self.params['pole_w'],self.params['pole_l'],\
                                      linewidth=.5,edgecolor='red',facecolor='red')
        self.cart = patches.Rectangle((state[0]-self.params['cart_w']/2,0),\
                                       self.params['cart_w'],self.params['cart_h'],\
                                      linewidth=.5,edgecolor='blue',facecolor='blue')
        self.recs = []
        self.recs.append(self.pole)
        self.recs.append(self.cart)
        for i in range(len(self.obs)):
            x, y = self.obs[i]
            obs = patches.Rectangle((x-self.params['obs_w']/2,y-params['obs_h']/2),\
                                       self.params['obs_w'],self.params['obs_h'],\
                                      linewidth=.5,edgecolor='black',facecolor='black')
            self.recs.append(obs)
            ax.add_patch(obs)
        # transform pole according to state
        t = mpl.transforms.Affine2D().rotate_deg_around(state[0], self.params['cart_h'], \
                                                        -state[2]/np.pi * 180) + ax.transData
        self.pole.set_transform(t)
        ax.add_patch(self.pole)
        ax.add_patch(self.cart)
        return self.recs
    def _animate(self, i):
        ax = plt.gca()
        ax.set_xlim(-40, 40)
        ax.set_ylim(-20, 20)
        state = self.states[i]
        self.recs[0].set_xy((state[0]-self.params['pole_w']/2,self.params['cart_h']))
        t = mpl.transforms.Affine2D().rotate_deg_around(state[0], self.params['cart_h'], \
                                                        -state[2]/np.pi * 180) + ax.transData
        self.recs[0].set_transform(t)
        self.recs[1].set_xy((state[0]-self.params['cart_w']/2,params['cart_h']))
        # print location of cart
        return self.recs


 
    def animate(self, states, actions, obstacles):
        '''
        given a list of states, actions and obstacles, animate the robot
        '''
        # transform the waypoint states and actions into trajectory
        traj = []
        for i in range(len(states)-1):
            print('state: %d, remaining: %d' % (i, len(states)-i))
            s = states[i]
            action = actions[i]
            sT = states[i+1]
            # propogate until reaching next state
            while True:
                traj.append(s)
                #print("porpagating...")
                #print(s)
                #print('st:')
                #print(sT)
                s = self.system.propagate(s, action, 1, self.params['integration_step'])
                if np.linalg.norm(s-sT) == 0.:
                    break
        traj = np.array(traj)
        print("animating...")
        # animate
        self.states = traj
        self.obs = obstacles
        print(len(self.states))
        ani = animation.FuncAnimation(plt.gcf(), self._animate, range(1, len(self.states)),
                                      interval=self.dt*10, blit=True, init_func=self._init,
                                      repeat=True)
        return ani


In [ ]:
params = {}
params['pole_l'] = 2.5
params['pole_w'] = 0.1
params['cart_w'] = 1.
params['cart_h'] = 0.5
params['obs_w'] = 4
params['obs_h'] = 4
params['integration_step'] = 0.002
system = CartPole(obs_list)
vis = CartPoleVisualizer(system, params)
states, actions, costs = solution
anim = vis.animate(states, actions, obs_list)
HTML(anim.to_html5_video())

In [ ]:
print(obs_list)